In [1]:
from sdv.sequential import PARSynthesizer
from sdv.metadata import SingleTableMetadata
from sdv.evaluation.single_table import evaluate_quality, get_column_plot

import pandas as pd

In [2]:
metadata = SingleTableMetadata()

# Detect metadata from CSV file
metadata.detect_from_csv(filepath='datasets/baseline.csv')

metadata.update_column(
    column_name='user_id',
    sdtype='id',
    #pii='False'
)

metadata.update_column(
    column_name='time',
    sdtype='numerical',
)

metadata.set_sequence_key(column_name='user_id')
metadata.set_sequence_index(column_name='time')

metadata.validate()

print(metadata)

{
    "sequence_key": "user_id",
    "METADATA_SPEC_VERSION": "SINGLE_TABLE_V1",
    "sequence_index": "time",
    "columns": {
        "time": {
            "sdtype": "numerical"
        },
        "user_id": {
            "sdtype": "id"
        },
        "session_id": {
            "sdtype": "numerical"
        },
        "acc_var": {
            "sdtype": "numerical"
        },
        "hr": {
            "sdtype": "numerical"
        },
        "rmssd": {
            "sdtype": "numerical"
        },
        "sdnn": {
            "sdtype": "numerical"
        },
        "temp": {
            "sdtype": "numerical"
        },
        "eda": {
            "sdtype": "numerical"
        },
        "eda_freq": {
            "sdtype": "numerical"
        },
        "bm": {
            "sdtype": "numerical"
        },
        "location": {
            "sdtype": "categorical"
        },
        "concentration": {
            "sdtype": "categorical"
        }
    }
}


In [7]:
data = pd.read_csv("datasets/baseline.csv", delimiter=',')

data.head()# Create the synthesizer
synthesizer = PARSynthesizer(
    metadata, # required
    enforce_min_max_values=False,
    enforce_rounding=False,
    epochs=180,
    verbose=True,
    cuda=True,
)

# Train the synthesizer
# synthesizer.fit(data)
# synthesizer.save(filepath='my_synthesizer.pkl')

# or load one
synthesizer = PARSynthesizer.load(filepath='my_synthesizer.pkl')

# Generate synthetic data
synthetic_data = synthesizer.sample(num_sequences=10)

  0%|          | 0/10 [00:00<?, ?it/s]/home/e/emandan/mlenv/lib/python3.10/site-packages/torch/nn/modules/rnn.py:1133: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Triggered internally at ../aten/src/ATen/native/cudnn/RNN.cpp:1424.)
  result = _VF.gru(input, hx, self._flat_weights, self.bias, self.num_layers,
 10%|█         | 1/10 [10:51<1:37:47, 651.95s/it]/home/e/emandan/mlenv/lib/python3.10/site-packages/torch/nn/modules/rnn.py:1133: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Triggered internally at ../aten/src/ATen/native/cudnn/RNN.cpp:1424.)
  result = _VF.gru(input, hx, self._flat_weights, self.bias, self.num_layer

In [11]:
synthetic_data.to_csv('synthetic_data_timeseries1.csv', index=False)

In [12]:
quality_report = evaluate_quality(
    data,
    synthetic_data,
    metadata)

Generating report ...

(1/2) Evaluating Column Shapes: |██████████| 13/13 [00:03<00:00,  4.16it/s]|
Column Shapes Score: 48.2%

(2/2) Evaluating Column Pair Trends: |██████████| 78/78 [00:44<00:00,  1.77it/s]|
Column Pair Trends Score: 77.12%

Overall Score (Average): 62.66%



In [19]:
fig = get_column_plot(
    real_data=data,
    synthetic_data=synthetic_data,
    column_name='concentration',
    metadata=metadata,
)
    
fig.show()